### Gather data 
    Online data (Twitter and URL)
    Stored files(txt and csv files)
    

In [15]:
import pandas as pd
import json
import csv
import re
import tweepy
from tweepy import OAuthHandler
import time
from urllib import request

In [12]:
#download image prediction file in url programmatically
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
 
def download_txturl(txturl):
    response = request.urlopen(txturl)
    txt = response.read()
    txtstr = str(txt)
    lines = txtstr.split('\\n')
    dest = r'image_predictions.tsv'
    op = open(dest, 'w')
    for line in lines:
        op.write(line + '\n')
    op.close()

download_txturl(url)


In [ ]:
consumer_key = 'jbL7N3K4eXgD1zhDfeS2TcnFr'
consumer_secret = 'XeIpJqZ4i8WAolHVCNBqGtILZKY1J0TVJw8GkpW9jDRthls1qn'
access_token = '495295426-3dc2rEDWSI8AX8NxYCpIRV9tk83EwocP7RzbraDC'
access_token_secret = 'zsulnNzoCcHRURBOkfVkb1tLcJPfYinSwfJBpn9Nam1FV'


# search_favorites(consumer_key, consumer_secret, access_token, access_token_secret):
#create authentication for accessing twitter
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#initialize Tweepy API
api = tweepy.API(auth, wait_on_rate_limit =True, wait_on_rate_limit_notify = True)
tweet_ids = df_archive.tweet_id


In [ ]:
count = 0
fails_dict = {}
#save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    
    #loop through each tweet id filtered by the given archive file and write to outfile
    for tweet_id in tweet_ids:
        count += 1        
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n')
            
        except tweepy.TweepError as e:
            fails_dict[tweet_id] = e
            pass

In [147]:
df_new = pd.DataFrame(columns=['tweet_id', 'text','retweet_count', 'favorite_count'])
with open('tweet_json.txt') as f:
    for line in f:
        status  = json.loads(line)
        tweet_id = status['id_str']
        text = status['full_text']
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        df_new = df_new.append(pd.DataFrame([[tweet_id, text, retweet_count, favorite_count]],
                                        columns=['tweet_id', 'text', 'retweet_count', 'favorite_count']))
df_new = df_new.reset_index(drop=True)

In [148]:
df_new.head()

,tweet_id,text,retweet_count,favorite_count
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,8193,37559
1,892177421306343426,This is Tilly. She's just checking pup on you....,6060,32284
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,4008,24325
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,8349,40908
4,891327558926688256,This is Franklin. He would like you to stop ca...,9041,39096


In [149]:
df_archive = pd.read_csv('twitter-archive-enhanced-2.csv')
df_image = pd.read_csv('image-predictions-3.tsv',  delimiter='\t')

### Assess


### Quality
archive table

missing data for retweets and likes
incorrect ratings data and unnusable values
    75 should be 9.75
    1776, 420, high values will skew analysis
incorrect data types 
    ID should be a string
    ratings should in floats 
    favorite counts and retweet counts should be intergers
    timestamp should be a datetime object
    category
unnecessary data
Retweet rows need to be removed
Rows without images need to be removed
    

image table
Results that are not dogs should be removed


### Tidiness 
archive table
the dog type should be added to the other table
the denominator rating is reduntant and should be removed 
dog stage is one variable and should therefore be one column
Tweets that contain ratings for multiple dogs should be removed

image data
There should be one dog breed for each entry



## Clean

### Missing data:
    The likes and retweet counts will be added. The archive and new dataframes are combined with an inner join. This will exclude values from the archive that are no longer present. This will be easier if the tweet IDs are strings. 
    

In [178]:
df_new2 = df_new.copy()
df_archive2 = df_archive.copy()
df_image2 = df_image.copy()

In [227]:
df_new2.tweet_id = df_new2['tweet_id'].astype('str')
df_archive2.tweet_id = df_archive2.tweet_id.astype('str')
df_new2.head()

,tweet_id,text,retweet_count,favorite_count
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,8193,37559
1,892177421306343426,This is Tilly. She's just checking pup on you....,6060,32284
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,4008,24325
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,8349,40908
4,891327558926688256,This is Franklin. He would like you to stop ca...,9041,39096


In [277]:
df_clean = df_archive2.merge(df_new2, how= 'inner', on = 'tweet_id')
df_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text_x,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,text_y,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,This is Phineas. He's a mystical boy. Only eve...,8193,37559
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,This is Tilly. She's just checking pup on you....,6060,32284
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,This is Archie. He is a rare Norwegian Pouncin...,4008,24325


### Unnecessary rows and columns


#### Retweets and rows are removed by removing rows that contain a value in retweet status and in reply to columns and those that do not contain photos. 

In [278]:
df_clean = df_clean[(df_clean['retweeted_status_id'].isna()) & (df_clean['in_reply_to_status_id'].isna())]


In [279]:
# the old text will be replaced by my newly collected text as it will be used later and I want to be sure of its correctness
df_clean.drop('text_x', axis = 1, inplace = True)

In [280]:
df_clean.rename(columns = {'text_y': 'text'}, inplace = True)

In [281]:
df_clean = df_clean[df_clean.text.str.contains("https://")]

In [282]:
df_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,text,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,This is Phineas. He's a mystical boy. Only eve...,8193,37559
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,This is Tilly. She's just checking pup on you....,6060,32284
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,This is Archie. He is a rare Norwegian Pouncin...,4008,24325
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,This is Darla. She commenced a snooze mid meal...,8349,40908
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,This is Franklin. He would like you to stop ca...,9041,39096


#### Unnecessary columns are removed.

In [283]:
df_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'rating_denominator', 'in_reply_to_status_id', 'in_reply_to_user_id'],  axis =1, inplace = True )

In [284]:
# check for any remaing text columns containing RT. The resulting rows are not retweets.
df_clean[df_clean.text.str.contains('RT ')]

,tweet_id,timestamp,source,expanded_urls,rating_numerator,name,doggo,floofer,pupper,puppo,text,retweet_count,favorite_count
1268,708400866336894977,2016-03-11 21:15:02 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",https://vine.co/v/iHFqnjKVbIQ,12,None,None,None,None,None,RT if you are as ready for summer as this pup ...,10621,16523
1632,683515932363329536,2016-01-03 05:11:12 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",https://vine.co/v/ibvnzrauFuV,11,None,None,None,None,None,HEY PUP WHAT'S THE PART OF THE HUMAN BODY THAT...,3075,7617
1841,675489971617296384,2015-12-12 01:38:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/675489971...,10,None,None,None,None,None,RT until we find this dog. Clearly a cool dog ...,621,1322
1972,672622327801233409,2015-12-04 03:43:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/672622327...,12,None,None,None,pupper,None,This lil pupper is sad because we haven't foun...,500,1305


#### Stage column is created by combining the stage columns. They are then removed.

In [285]:
df_clean['stage'] = df_clean.text.str.findall(r'(doggo|floofer|pupper|puppo)')


In [286]:
df_clean.stage

0             []
1             []
2             []
3             []
4             []
5             []
6             []
7             []
8             []
9        [doggo]
10            []
11            []
12       [puppo]
13            []
14       [puppo]
15            []
16            []
17            []
18            []
19            []
20            []
21            []
22            []
23            []
24            []
25            []
26            []
27            []
28      [pupper]
30            []
          ...   
2307          []
2308          []
2309          []
2310          []
2311          []
2312          []
2313          []
2314          []
2315          []
2316          []
2317          []
2318          []
2319          []
2320          []
2321          []
2322          []
2323          []
2324          []
2325          []
2326          []
2327          []
2328          []
2329          []
2330          []
2331          []
2332          []
2333          []
2334          

In [287]:
# the new stage column is a list. The few columns with double entries are separated with a comma and all values become strings. 
df_clean['stage'] = df_clean['stage'].str.join(',')


In [288]:
#replecated stages are removed
df_clean['stage'].replace('pupper,pupper', 'pupper', inplace =True) 
df_clean['stage'].replace('pupper,doggo,doggo', 'pupper,doggo', inplace = True) 
df_clean['stage'].replace('pupper,pupper,pupper', 'pupper', inplace = True) 

In [289]:
#test
df_clean.stage.value_counts()

                1738
pupper           237
doggo             75
puppo             27
doggo,pupper       5
floofer            4
pupper,doggo       2
puppo,doggo        2
Name: stage, dtype: int64

Remove columns

In [348]:
df_clean2 = df_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis =1)

### Correction of ratings
#### The ratings are very important to the analysis. The text is queried to recollect the ratings. 

In [349]:
#A regular expression is used to find all ratings
df_clean2['rating'] = df_clean2['text'].str.findall(r'(\d+/10)')
df_clean2.rating.value_counts()

[12/10]                 483
[10/10]                 423
[11/10]                 411
[13/10]                 286
[9/10]                  152
[8/10]                   96
[7/10]                   50
[14/10]                  39
[5/10]                   32
[6/10]                   32
[3/10]                   19
[4/10]                   14
[]                       12
[2/10]                    9
[1/10]                    4
[10/10, 7/10]             3
[11/10, 8/10]             2
[10/10, 4/10]             2
[4/10, 13/10]             1
[75/10]                   1
[420/10]                  1
[10/10, 10/10]            1
[10/10, 7/10, 12/10]      1
[8/10, 11/10]             1
[5/10, 8/10]              1
[0/10]                    1
[10/10, 11/10]            1
[27/10]                   1
[9/10, 2/10]              1
[26/10]                   1
[12/10, 11/10]            1
[11/10, 10/10]            1
[10/10, 6/10]             1
[5/10, 10/10]             1
[10/10, 5/10]             1
[1776/10]           

In [350]:
# rows with multiple ratings for no ratings are removed

df_clean2 = df_clean2[(df_clean2['rating'].str.len() < 2) & (df_clean2['rating'].str.len() > 0)]
df_clean2.head(1)

,tweet_id,timestamp,source,expanded_urls,rating_numerator,name,text,retweet_count,favorite_count,stage,rating
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/892420643...,13,Phineas,This is Phineas. He's a mystical boy. Only eve...,8193,37559,,[13/10]
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/892177421...,13,Tilly,This is Tilly. She's just checking pup on you....,6060,32284,,[13/10]
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/891815181...,12,Archie,This is Archie. He is a rare Norwegian Pouncin...,4008,24325,,[12/10]
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/891689557...,13,Darla,This is Darla. She commenced a snooze mid meal...,8349,40908,,[13/10]
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/891327558...,12,Franklin,This is Franklin. He would like you to stop ca...,9041,39096,,[12/10]


In [351]:
#the value is selected from the list so that it is no more in list format
df_clean2['rating'] = df_clean2['rating'].apply(lambda x: x[0])

In [352]:
df_clean.head()

,tweet_id,timestamp,source,expanded_urls,rating_numerator,name,doggo,floofer,pupper,puppo,text,retweet_count,favorite_count,stage
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/892420643...,13,Phineas,None,None,None,None,This is Phineas. He's a mystical boy. Only eve...,8193,37559,
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/892177421...,13,Tilly,None,None,None,None,This is Tilly. She's just checking pup on you....,6060,32284,
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/891815181...,12,Archie,None,None,None,None,This is Archie. He is a rare Norwegian Pouncin...,4008,24325,
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/891689557...,13,Darla,None,None,None,None,This is Darla. She commenced a snooze mid meal...,8349,40908,
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/891327558...,12,Franklin,None,None,None,None,This is Franklin. He would like you to stop ca...,9041,39096,


In [353]:
# The rating column is split again to extract the numerator 
#df_clean2.rating = df_clean2.rating.str.split('/')

In [354]:
# The numerator is selected
#df_clean2['rating'] = df_clean2['rating'].apply(lambda x: x[0:2])
df_clean2.rating = df_clean2['rating'].str.extract('(^\d+)')

In [355]:
df_clean2.rating = df_clean2.rating.astype(float)

In [356]:
# was curious about the differences between my rating results and the given rating result. 
df_clean2.query('rating != rating_numerator')

,tweet_id,timestamp,source,expanded_urls,rating_numerator,name,text,retweet_count,favorite_count,stage,rating
1050,740373189193256964,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/740373189...,9,None,"After so many requests, this is Bretagne. She ...",13988,36059,,14.0
1147,722974582966214656,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/722974582...,4,None,Happy 4/20 from the squad! 13/10 for all https...,1641,4220,,13.0
1184,716439118184652801,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/716439118...,50,Bluebert,This is Bluebert. He just saw that both #Final...,219,2439,,11.0
1644,682962037429899265,2016-01-01 16:30:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/682962037...,7,Darrel,This is Darrel. He just robbed a 7/11 and is i...,17583,38245,,10.0
2316,666287406224695296,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",https://twitter.com/dog_rates/status/666287406...,1,an,This is an Albanian 3 1/2 legged Episcopalian...,63,142,,9.0


In [357]:
#I am now comfortable removing the rating numerator column
df_clean2.drop('rating_numerator', axis =1, inplace =True)

### Make one column for breeds


In [358]:
#new data frames are created that will designate the most likely dog breed for each tweet. Predictions where no result was a 
#dog breed will be removed and the breed collumn will be a null value for the tweets with no resonable prediction. 
df_1 = df_image2.query('p1_dog == True')
df_2 = df_image2.query('p1_dog != True & p2_dog == True')
df_3 = df_image2.query('p1_dog != True & p2_dog != True & p3_dog == True')

df_1['breed'] = df_1['p1']
df_2['breed'] = df_2['p2']
df_3['breed'] = df_3['p3']
len(df_1) + len(df_2) + len(df_3)
len(df_image)

C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

2075

In [361]:
# The new dataframes are concatenated into one. 
image_clean = pd.concat([df_1, df_2, df_3], axis=0, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
# All columns except the new breed column and the twitter ID are removed. 
image_clean = image_clean[['tweet_id','breed']]

In [363]:
# The data types are changed to strings
image_clean = image_clean.astype(str, copy=True)
# The dataframes are merged so that there is now a breed column in the main dataframe.
df_clean2.merge(image_clean, how = 'left', on = 'tweet_id')
df_clean2.info()
df_clean2.tweet_id.duplicated().any()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2056 entries, 0 to 2336
Data columns (total 10 columns):
tweet_id          2056 non-null object
timestamp         2056 non-null object
source            2056 non-null object
expanded_urls     2056 non-null object
name              2056 non-null object
text              2056 non-null object
retweet_count     2056 non-null object
favorite_count    2056 non-null object
stage             2056 non-null object
rating            2056 non-null float64
dtypes: float64(1), object(9)
memory usage: 176.7+ KB


False

In [347]:
Change data types 

SyntaxError: invalid syntax (<ipython-input-347-d7d4a1ebf0ae>, line 1)

In [366]:
#datatypes are changed to allow us to analyze the data.
df_clean2.retweet_count = df_clean2.retweet_count.astype(int)
df_clean2.favorite_count = df_clean2.favorite_count.astype(int)
df_clean2.stage = df_clean2.stage.astype('category')
#df_clean.rating = df_clean.rating.astype('category')
df_clean2.timestamp = pd.to_datetime(df_clean2['timestamp'])


In [368]:
df_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2056 entries, 0 to 2336
Data columns (total 10 columns):
tweet_id          2056 non-null object
timestamp         2056 non-null datetime64[ns, UTC]
source            2056 non-null object
expanded_urls     2056 non-null object
name              2056 non-null object
text              2056 non-null object
retweet_count     2056 non-null int32
favorite_count    2056 non-null int32
stage             2056 non-null category
rating            2056 non-null float64
dtypes: category(1), datetime64[ns, UTC](1), float64(1), int32(2), object(5)
memory usage: 146.9+ KB


#### I am interested in exploring differences between days of the week so I added a column with this data. 

In [369]:
import datetime as dt
df_clean2['day_of_week'] = df_clean2['timestamp'].dt.dayofweek

# Analyse